# data set 
Foi pego no uci, convertido para csv com auxilio de script


In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from statistics import mean 
from collections import Counter
from sklearn.metrics import confusion_matrix
import  matplotlib.pyplot as plt
import seaborn as sn

In [2]:
headers = list(pd.read_csv("EGG_EyeStateHeaders.csv") )

In [3]:
data = pd.read_csv("EEG_EyeState.csv", names=headers, index_col=False, skiprows = [0]) 
# Preview the first 5 lines of the loaded data 
data.head()

,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,eyeDetection
0,4329.23,4009.23,4289.23,4148.21,4350.26,4586.15,4096.92,4641.03,4222.05,4238.46,4211.28,4280.51,4635.90,4393.85,0
1,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
2,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
3,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
4,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data, data.eyeDetection, test_size=0.3)                                                    
X_train = X_train.drop("eyeDetection", axis=1)
X_test = X_test.drop("eyeDetection", axis=1)

In [5]:
# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

Algoritmo diferente para a harmonica

In [6]:
from scipy.spatial import distance as dist_lib
def euclidean(instance1, instance2):
    return dist_lib.euclidean(instance1, instance2)

def canberra(instance1, instance2):
    return dist_lib.canberra(instance1, instance2)

def chebyshev(instance1, instance2):
    return dist_lib.chebyshev(instance1, instance2)

def correlation(instance1, instance2):
    return dist_lib.correlation(instance1, instance2)

def cosine(instance1, instance2):
    return dist_lib.cosine(instance1, instance2)

In [7]:
def get_neighbors(training_set, 
                  labels, 
                  test_instance, 
                  k, 
                  distance):
    """
    get_neighors calculates a list of the k nearest neighbors
    of an instance 'test_instance'.
    The list neighbors contains 3-tuples with  
    (index, dist, label),
    where:
    index    is the index from the training_set, 
    dist     is the distance between the test_instance 
             and the instance training_set[index]
    distance is a reference to a function used to calculate the 
             distances
    """
    distances = []
    for index, row in training_set.iterrows():
        dist = distance(test_instance, row)
        distances.append((row, dist, labels[index]))
    distances.sort(key=lambda x: x[1])
    neighbors = distances[:k]
    return neighbors


            
def vote_harmonic_weights(neighbors, all_results=True):
    class_counter = Counter()
    number_of_neighbors = len(neighbors)
    
    # para cada classe, somam-se os inversos de suas ordens
    for index in range(number_of_neighbors):
        class_counter[neighbors[index][2]] += 1/(index+1)
    labels, votes = zip(*class_counter.most_common())
    winner = class_counter.most_common(1)[0][0]
    votes4winner = class_counter.most_common(1)[0][1]
    if all_results:
        total = sum(class_counter.values(), 0.0)
        for key in class_counter:
             class_counter[key] /= total
        return winner, class_counter.most_common()
    else:
        return winner, votes4winner / sum(votes)
    
def knn_test_harmonic(trainset_data, trainset_labels, testset_data, knn_k, distance_function):
    print(testset_data.shape[0])
    count_row = 0
    df_prediction = pd.DataFrame(columns=['class'])
    for index, row in testset_data.iterrows():
        neighbors = get_neighbors(trainset_data, 
                                  trainset_labels, 
                                  row, 
                                  knn_k, 
                                  distance=distance_function)
        #print(vote_harmonic_weights(neighbors, all_results=True))
        predited_class = vote_harmonic_weights(neighbors, all_results=True)[0]
        df_prediction.loc[index] = predited_class
        count_row += 1
        if (count_row % 50 == 0):
            print(count_row, count_row/testset_data.shape[0])
    
    return df_prediction

In [8]:
weights = ['uniform', 'distance', 'harmonic']

distances = ['canberra', 'chebyshev', 'correlation', 'euclidean','cosine' ] #'mahalanobis'
V = np.cov(X_train)

def save_cm(cm, count_fold, knn_k, distance, weight_function):
    if count_fold == 'test' or count_fold == 'final':
        df_cm = cm
    else:
        count_fold = "fold" + str(count_fold)
        df_cm = pd.DataFrame(cm, range(2), range(2))
        
    plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
    plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
    sn.set(font_scale=1.4)
    sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g') # font size
    plt.savefig("/home/cadic/projetos-cadic/teste/{}/{}/{}/{}.png".format(weight_function, distance, knn_k,  count_fold))
    plt.clf()
    return df_cm

In [9]:
def cross_validation(knn_k, distance, weight_function):
    if (weight_function == 'harmonic'):
        kf = KFold(n_splits= 5, shuffle=True)
        accs = []
        count_fold = 0
        all_matrix = []
        for train_index, test_index in kf.split(X_train):
            fold_x_train, fold_x_test = X_train[X_train.index.isin(train_index)], X_train[X_train.index.isin(test_index)]
            fold_y_train, fold_y_test = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]
                
            print("Treino: " + str(X_train.shape[0]) + ", fold_treino: " + str(fold_x_train.shape[0]) + ", fold_test: "+ str(fold_x_test.shape[0]))
            
            #Train the model using the training sets
            if (distance == 'canberra'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, canberra)
            elif (distance == 'chebyshev'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'correlation'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'euclidean'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
            elif (distance == 'cosine'):
                fold_y_pred = knn_test_harmonic(fold_x_train, fold_y_train, fold_x_test, knn_k, euclidean)
                
            acc = metrics.accuracy_score(fold_y_test, fold_y_pred['class'].astype('int32'))
            # Model Accuracy, how often is the classifier correct?
            print("Accuracy:", acc)
            accs.append(acc)
            cm = confusion_matrix(fold_y_test, fold_y_pred['class'].astype('int32'))
            #print(cm)
            save_cm(cm, count_fold, knn_k, distance, weight_function)
            matrix = save_cm(cm, count_fold, knn_k, distance, weight_function)
            all_matrix.append(matrix)
            count_fold += 1
        
        mean_matrix = pd.concat(all_matrix).groupby(level=0).mean()
        save_cm(mean_matrix, 'final', knn_k, distance, weight_function)
        return [knn_k, distance, weight_function, mean(accs)]
            
    else:
        #Create KNN Classifier
        if distance == 'mahalanobis':
            knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
        else:
            knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)

        kf = KFold(n_splits= 5, shuffle=True)
        count_fold = 0
        accs = []
        all_matrix = []
        for train_index, test_index in kf.split(X_train):
            fold_x_train, fold_x_test = X_train[X_train.index.isin(train_index)], X_train[X_train.index.isin(test_index)]
            fold_y_train, fold_y_test = y_train[y_train.index.isin(train_index)], y_train[y_train.index.isin(test_index)]
            #Train the model using the training sets
            knn.fit(fold_x_train, fold_y_train)

            #Predict the response for test dataset
            fold_y_pred = knn.predict(fold_x_test)

            acc = metrics.accuracy_score(fold_y_test, fold_y_pred)
            # Model Accuracy, how often is the classifier correct?
            print("Accuracy:", acc)
            accs.append(acc)
            cm = confusion_matrix(fold_y_test, fold_y_pred)
            matrix = save_cm(cm, count_fold, knn_k, distance, weight_function)
            all_matrix.append(matrix)
            count_fold += 1
        
        mean_matrix = pd.concat(all_matrix).groupby(level=0).mean()
        save_cm(mean_matrix, 'final', knn_k, distance, weight_function)
        return [knn_k, distance, weight_function, mean(accs)]

In [10]:
def test(knn_k, distance, weight_function):
    #Create KNN Classifier
    if distance == 'mahalanobis':
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function, metric_params={'V': V})
    else:
        knn = KNeighborsClassifier(n_neighbors=knn_k, metric=distance, weights = weight_function)

    #Train the model using the training sets
    knn.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = knn.predict(X_test)

    cm_test = confusion_matrix(y_test, y_pred)
    save_cm(cm_test, 'test', knn_k, distance, weight_function)
    
    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [11]:
def select_bests_params(results):
    df_results = pd.DataFrame(results)
    return df_results.sort_values([3],  ascending=[0])

In [12]:
results = []
for weight_function in weights:
    for knn_k in range(3,16,2):
        for distance in distances:
            print("Função: {} - distancia: {} - K: {}".format(weight_function,distance, knn_k))
            results.append(cross_validation(knn_k, distance, weight_function))

ranking = select_bests_params(results)
print(ranking)

Função: uniform - distancia: canberra - K: 3
Accuracy: 0.959891230455
Accuracy: 0.955752212389
Accuracy: 0.953424657534
Accuracy: 0.949129852744
Accuracy: 0.952315134762
Função: uniform - distancia: chebyshev - K: 3
Accuracy: 0.945355191257
Accuracy: 0.957317073171
Accuracy: 0.95836131632
Accuracy: 0.950865051903
Accuracy: 0.945466939332
Função: uniform - distancia: correlation - K: 3
Accuracy: 0.953646898432
Accuracy: 0.966622162884
Accuracy: 0.955704697987
Accuracy: 0.961218836565
Accuracy: 0.960471567268
Função: uniform - distancia: euclidean - K: 3
Accuracy: 0.958587915818
Accuracy: 0.963673749143
Accuracy: 0.950608930988
Accuracy: 0.969209237229
Accuracy: 0.963382157124
Função: uniform - distancia: cosine - K: 3
Accuracy: 0.957774798928
Accuracy: 0.96540027137
Accuracy: 0.972696245734
Accuracy: 0.97323266987
Accuracy: 0.975911906401
Função: uniform - distancia: canberra - K: 5
Accuracy: 0.959128065395
Accuracy: 0.949140893471
Accuracy: 0.947615849563
Accuracy: 0.941460055096
Accur

Accuracy: 0.96
Accuracy: 0.951831750339
Accuracy: 0.953145917001
Accuracy: 0.95150273224
Função: distance - distancia: cosine - K: 7
Accuracy: 0.965445749827
Accuracy: 0.967828418231
Accuracy: 0.968493150685
Accuracy: 0.975711311589
Accuracy: 0.962691538974
Função: distance - distancia: canberra - K: 9
Accuracy: 0.936400541272
Accuracy: 0.950134770889
Accuracy: 0.948240165631
Accuracy: 0.933743169399
Accuracy: 0.954979536153
Função: distance - distancia: chebyshev - K: 9
Accuracy: 0.941860465116
Accuracy: 0.93537414966
Accuracy: 0.945307224848
Accuracy: 0.932478055368
Accuracy: 0.949550794748
Função: distance - distancia: correlation - K: 9
Accuracy: 0.959128065395
Accuracy: 0.959128065395
Accuracy: 0.958503401361
Accuracy: 0.956790123457
Accuracy: 0.955314827353
Função: distance - distancia: euclidean - K: 9
Accuracy: 0.952777777778
Accuracy: 0.956313993174
Accuracy: 0.955674949631
Accuracy: 0.948299319728
Accuracy: 0.953283683142
Função: distance - distancia: cosine - K: 9
Accuracy: 

1300 0.8928571428571429
1350 0.9271978021978022
1400 0.9615384615384616
1450 0.9958791208791209
Accuracy: 0.964285714286
Treino: 10486, fold_treino: 5839, fold_test: 1502
1502
50 0.033288948069241014
100 0.06657789613848203
150 0.09986684420772303
200 0.13315579227696406
250 0.16644474034620507
300 0.19973368841544606
350 0.23302263648468707
400 0.2663115845539281
450 0.2996005326231691
500 0.33288948069241014
550 0.36617842876165113
600 0.3994673768308921
650 0.43275632490013316
700 0.46604527296937415
750 0.4993342210386152
800 0.5326231691078562
850 0.5659121171770972
900 0.5992010652463382
950 0.6324900133155792
1000 0.6657789613848203
1050 0.6990679094540613
1100 0.7323568575233023
1150 0.7656458055925432
1200 0.7989347536617842
1250 0.8322237017310253
1300 0.8655126498002663
1350 0.8988015978695073
1400 0.9320905459387483
1450 0.9653794940079894
1500 0.9986684420772304
Accuracy: 0.975366178429
Treino: 10486, fold_treino: 5899, fold_test: 1442
1442
50 0.03467406380027739
100 0.069

800 0.547945205479452
850 0.5821917808219178
900 0.6164383561643836
950 0.6506849315068494
1000 0.684931506849315
1050 0.7191780821917808
1100 0.7534246575342466
1150 0.7876712328767124
1200 0.821917808219178
1250 0.8561643835616438
1300 0.8904109589041096
1350 0.9246575342465754
1400 0.958904109589041
1450 0.9931506849315068
Accuracy: 0.969178082192
Treino: 10486, fold_treino: 5861, fold_test: 1480
1480
50 0.033783783783783786
100 0.06756756756756757
150 0.10135135135135136
200 0.13513513513513514
250 0.16891891891891891
300 0.20270270270270271
350 0.23648648648648649
400 0.2702702702702703
450 0.30405405405405406
500 0.33783783783783783
550 0.3716216216216216
600 0.40540540540540543
650 0.4391891891891892
700 0.47297297297297297
750 0.5067567567567568
800 0.5405405405405406
850 0.5743243243243243
900 0.6081081081081081
950 0.6418918918918919
1000 0.6756756756756757
1050 0.7094594594594594
1100 0.7432432432432432
1150 0.777027027027027
1200 0.8108108108108109
1250 0.8445945945945946
1

500 0.3429355281207133
550 0.3772290809327846
600 0.411522633744856
650 0.4458161865569273
700 0.48010973936899864
750 0.51440329218107
800 0.5486968449931413
850 0.5829903978052127
900 0.6172839506172839
950 0.6515775034293553
1000 0.6858710562414266
1050 0.720164609053498
1100 0.7544581618655692
1150 0.7887517146776406
1200 0.823045267489712
1250 0.8573388203017832
1300 0.8916323731138546
1350 0.9259259259259259
1400 0.9602194787379973
1450 0.9945130315500685
Accuracy: 0.962962962963
Treino: 10486, fold_treino: 5909, fold_test: 1432
1432
50 0.034916201117318434
100 0.06983240223463687
150 0.10474860335195531
200 0.13966480446927373
250 0.17458100558659218
300 0.20949720670391062
350 0.24441340782122906
400 0.27932960893854747
450 0.31424581005586594
500 0.34916201117318435
550 0.3840782122905028
600 0.41899441340782123
650 0.45391061452513964
700 0.4888268156424581
750 0.5237430167597765
800 0.5586592178770949
850 0.5935754189944135
900 0.6284916201117319
950 0.6634078212290503
1000 

...

850 0.5809979494190021
900 0.6151742993848257
950 0.6493506493506493
1000 0.683526999316473
1050 0.7177033492822966
1100 0.7518796992481203
1150 0.7860560492139439
1200 0.8202323991797676
1250 0.8544087491455913
1300 0.8885850991114149
1350 0.9227614490772386
1400 0.9569377990430622
1450 0.9911141490088858
Accuracy: 0.96035543404
Função: harmonic - distancia: correlation - K: 13
Treino: 10486, fold_treino: 5856, fold_test: 1485
1485
50 0.03367003367003367
100 0.06734006734006734
150 0.10101010101010101
200 0.13468013468013468
250 0.16835016835016836
300 0.20202020202020202
350 0.2356902356902357
400 0.26936026936026936
450 0.30303030303030304
500 0.3367003367003367
550 0.37037037037037035
600 0.40404040404040403
650 0.4377104377104377
700 0.4713804713804714
750 0.5050505050505051
800 0.5387205387205387
850 0.5723905723905723
900 0.6060606060606061
950 0.6397306397306397
1000 0.6734006734006734
1050 0.7070707070707071
1100 0.7407407407407407
1150 0.7744107744107744
1200 0.80808080808080

250 0.1721763085399449
300 0.2066115702479339
350 0.24104683195592286
400 0.27548209366391185
450 0.30991735537190085
500 0.3443526170798898
550 0.3787878787878788
600 0.4132231404958678
650 0.4476584022038568
700 0.4820936639118457
750 0.5165289256198347
800 0.5509641873278237
850 0.5853994490358126
900 0.6198347107438017
950 0.6542699724517906
1000 0.6887052341597796
1050 0.7231404958677686
1100 0.7575757575757576
1150 0.7920110192837465
1200 0.8264462809917356
1250 0.8608815426997245
1300 0.8953168044077136
1350 0.9297520661157025
1400 0.9641873278236914
1450 0.9986225895316805
Accuracy: 0.964187327824
Treino: 10486, fold_treino: 5915, fold_test: 1426
1426
50 0.03506311360448808
100 0.07012622720897616
150 0.10518934081346423
200 0.1402524544179523
250 0.1753155680224404
300 0.21037868162692847
350 0.24544179523141654
400 0.2805049088359046
450 0.3155680224403927
500 0.3506311360448808
550 0.3856942496493689
600 0.42075736325385693
650 0.45582047685834504
700 0.4908835904628331
750 

1300 0.895933838731909
1350 0.9303928325292902
1400 0.9648518263266712
1450 0.9993108201240524
Accuracy: 0.97105444521
Treino: 10486, fold_treino: 5858, fold_test: 1483
1483
50 0.033715441672285906
100 0.06743088334457181
150 0.10114632501685772
200 0.13486176668914363
250 0.16857720836142953
300 0.20229265003371544
350 0.23600809170600134
400 0.26972353337828725
450 0.30343897505057316
500 0.33715441672285906
550 0.37086985839514497
600 0.4045853000674309
650 0.4383007417397168
700 0.4720161834120027
750 0.5057316250842886
800 0.5394470667565745
850 0.5731625084288604
900 0.6068779501011463
950 0.6405933917734322
1000 0.6743088334457181
1050 0.708024275118004
1100 0.7417397167902899
1150 0.7754551584625758
1200 0.8091706001348618
1250 0.8428860418071477
1300 0.8766014834794336
1350 0.9103169251517195
1400 0.9440323668240054
1450 0.9777478084962913
Accuracy: 0.964261631827
Treino: 10486, fold_treino: 5865, fold_test: 1476
1476
50 0.03387533875338753
100 0.06775067750677506
150 0.101626

850 0.5817932922655715
900 0.6160164271047228
950 0.6502395619438741
1000 0.6844626967830253
1050 0.7186858316221766
1100 0.7529089664613279
1150 0.7871321013004792
1200 0.8213552361396304
1250 0.8555783709787816
1300 0.8898015058179329
1350 0.9240246406570842
1400 0.9582477754962354
1450 0.9924709103353867
Accuracy: 0.965092402464
Treino: 10486, fold_treino: 5884, fold_test: 1457
1457
50 0.03431708991077557
100 0.06863417982155114
150 0.1029512697323267
200 0.13726835964310227
250 0.17158544955387783
300 0.2059025394646534
350 0.24021962937542896
400 0.27453671928620454
450 0.3088538091969801
500 0.34317089910775567
550 0.37748798901853126
600 0.4118050789293068
650 0.4461221688400824
700 0.4804392587508579
750 0.5147563486616334
800 0.5490734385724091
850 0.5833905284831846
900 0.6177076183939602
950 0.6520247083047358
1000 0.6863417982155113
1050 0.7206588881262869
1100 0.7549759780370625
1150 0.789293067947838
1200 0.8236101578586136
1250 0.8579272477693891
1300 0.8922443376801648


<Figure size 432x288 with 0 Axes>

In [13]:
best_knn_k, best_distance, best_weight_function, best_avg_acc = list(ranking.head(1).itertuples(index=False, name=None))[0]

In [14]:
best_knn_k, best_distance, best_weight_function

(3, 'cosine', 'distance')

In [15]:
test(best_knn_k, best_distance, best_weight_function)

Accuracy: 0.973520249221


<Figure size 432x288 with 0 Axes>

In [ ]:
#salvar o ranking em csv


In [16]:
type(y_test)

#acc = metrics.accuracy_score(fold_y_test, fold_y_pred)

pandas.core.series.Series

In [17]:
ranking.to_csv('/home/cadic/projetos-cadic/teste/ranking_dataframe.csv', index = True, header=True)